# CylSweep.ipynb
This notebook demonstrates the use of the `swapy` package to produce a set of wakefield Green's functions a cylindrical dielectric-lined waveguide

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from swapy import get_bunch
from swapy.structures import Cylinder

## Define waveguide properties
First we'll define arrays of geometric and physical waveguide parameters:

In [ ]:
# Define geometric parameters
a = 1. # cm e-2
n_xi = 10
Xi = np.linspace(1.e-3, 1., n_xi, endpoint=False)

# Define physical parameters
mu = 1.0
n_eps = 8
eps = np.linspace(1.5, 5.0, n_eps, endpoint=True)

## Define evaluation conditions
Next we'll define the evaluation domain and an array of ratios between source and inner radii:

In [ ]:
Lc = 10.    # Length of the cylinder (cm)
nZ = 1000   # Number of longitudinal evaluation values
nR = 100    # Number of radial evaluation values
nr0 = 10    # Number of source radius values
n_m = 3     # Number of modes
n_lam = 5   # Number of roots per mode

# Define radial ratios & longitudinal evaluation values
r0_rat = np.linspace(1.e-3, 1., nr0, endpoint=False)
R_rat = np.linspace(0., 1., nR, endpoint=False)
Z = np.linspace(0., Lc, nZ, endpoint=True)

# Generate Green's function data
Using the parameters defined above, we can now compute longitudinal and transverse Green's functions:

In [ ]:
GL = np.zeros((n_xi, n_eps, nr0, nR, nZ, n_m*n_lam))
GT = np.zeros((n_xi, n_eps, nr0, nR, nZ, n_m*n_lam))

# Loop over epsilon & xi
for e in range(n_eps):
    epsilon = eps[e]
    for n in range(n_xi):

        # Compute radial values in ratio to a & b
        b = Xi[n] * a
        r0 = b * r0_rat
        R = b * R_rat

        # Construct cylindrical waveguide
        cyl = Cylinder(a, b, mu, epsilon)

        # Loop over mode orders
        for m in range(n_m):

            # Compute Green's function amplitudes 
            GLm, GTm = cyl.greens(m, r0, R, Z, n_lam)
            GL[n, e, ..., m*n_lam:(m+1)*n_lam] = GLm
            GT[n, e, ..., m*n_lam:(m+1)*n_lam] = GTm

## Plot a sample of the data
### Choose sample for plotting
First we'll randomly select a set of waveguide and source parameters ($\xi$, $\epsilon$, and $r_0$):

In [ ]:
# Randomly choose waveguide & source parameters
rIDs = [
    np.random.choice(n_xi),
    np.random.choice(n_eps),
    np.random.choice(nr0)
]
r_xi = Xi[rIDs[0]]
r_eps = eps[rIDs[1]]
r_r0 = (r_xi * a * r0_rat[rIDs[2]])

then select a mode and index the corresponding Green's function data:

In [ ]:
# Mode for plotting
mode = 1

# Green's functions for parameter & mode choices
plot_GL = GL[*rIDs, ..., mode*n_lam:(mode+1)*n_lam]
plot_GT = GT[*rIDs, ..., mode*n_lam:(mode+1)*n_lam]

### Plot Green's functions
Now we'll plot the corresponding modal Green's functions:

In [ ]:
#
ex = [0., Lc*10., 0., r_xi*a*10]
plot_args = {
    "cmap": 'gray',
    "origin": 'lower',
    "extent": ex,
    "aspect": .3*ex[1]/ex[-1],
    "vmin": min([plot_GL.min(), plot_GT.min()]),
    "vmax": max([plot_GL.max(), plot_GT.max()])
}

#
fig, axs = plt.subplots(n_lam, 2, figsize=(18, 5*n_m))
fig.suptitle(
    r"Green's Functions, $G_{:s}$".format("{"+"{:d}".format(mode)+",\lambda}")+
    "\n"+r"$\xi={:.2f}$, $\epsilon={:.2f}$, $r_0={:.2f}$ cm".format(r_xi, r_eps, r_r0),
    fontsize=18, y=.95
)
axs[0,0].set_title(r"Longitudinal", fontsize=14)
axs[0,1].set_title(r"Transverse", fontsize=14)
for m in range(n_lam):
    axs[m,0].imshow(plot_GL[..., m], **plot_args)
    axs[m,1].imshow(plot_GT[..., m], **plot_args)
    for ax in axs[m]:
        ax.set_ylabel(r"$R$ (mm)", fontsize=14)
        ax.tick_params(labelsize=14)
        lam_label = ax.text(.99*ex[1], .97*ex[3], r"$\lambda={:d}$".format(m+1), color='k', fontsize=12, va='top', ha='right')
        lam_label.set_bbox(dict(facecolor='white', alpha=1.))
for ax in axs[m]:
    ax.set_xlabel(r"$Z$ (mm)", fontsize=14)

### Plot wakefields
Finally, we'll define a source bunch and convolve with the above Green's functions to compute and plot modal wake fields:

In [ ]:
# Define a source bunch
sigma = .5
Q = 100.e-9 * 1e-6
bunch = get_bunch(Q, Z, sigma)

# Compute wakefields
plot_WL = np.array([np.convolve(bunch, G)[0:nZ] for G in plot_GL.sum(axis=2)]) / Q
plot_WT = np.array([np.convolve(bunch, G)[0:nZ] for G in plot_GT.sum(axis=2)]) / Q

In [ ]:
plot_args = {
    "cmap": 'gray',
    "origin": 'lower',
    "extent": ex,
    "aspect": .45*ex[1]/ex[-1],
    "vmin": min([plot_WL.min(), plot_WT.min()]),
    "vmax": max([plot_WL.max(), plot_WT.max()])
}

#
fig, axs = plt.subplots(1, 2, figsize=(18, 5))
fig.suptitle(
    r"Wake Fields, $W(R,Z)$"+
    "\n"+r"$\xi={:.2f}$, $\epsilon={:.2f}$, $r_0={:.2f}$ cm".format(r_xi, r_eps, r_r0),
    fontsize=18, y=.99
)
axs[0].set_title(r"Longitudinal", fontsize=14)
axs[1].set_title(r"Transverse", fontsize=14)
axs[0].imshow(plot_WL, **plot_args)
axs[1].imshow(plot_WT, **plot_args)
for ax in axs:
    ax.tick_params(labelsize=14)
for ax in axs:
    ax.set_ylabel(r"$R$ (mm)", fontsize=14)
    ax.set_xlabel(r"$Z$ (mm)", fontsize=14)

In [ ]:
Rind = -1
plot_R = R[Rind]
plot_rho = (bunch/bunch.sum())*1.e2

fig, axs = plt.subplots(1, 2, figsize=(18,5))
fig.suptitle(
    r"Wake Fields, $W(R={:.2f}$ cm, $Z)$".format(plot_R)+
    "\n"+r"$\xi={:.2f}$, $\epsilon={:.2f}$, $r_0={:.2f}$ cm".format(r_xi, r_eps, r_r0),
    fontsize=18, y=1.1
)
axs[0].set_title(r"Longitudinal", fontsize=14)
axs[1].set_title(r"Transverse", fontsize=14)
axs[0].plot(Z, plot_WL[-1], 'k', lw=2.5)
axs[1].plot(Z, plot_WT[-1], 'k', lw=2.5)
axts = []
for ax in axs:
    axts.append(ax.twinx())
    axts[-1].plot(Z, plot_rho, c='g', lw=2.5)
    axts[-1].set_ylabel('Charge Density ($\%$)', fontsize=14, color='g')
    axts[-1].axis([Z.min(), Z.max(), 0., 2*plot_rho.max()])
    axts[-1].tick_params(labelsize=14, labelcolor='g')
axs[0].axis([0, Z.max(), 4.1*plot_WL.min(), 1.1*plot_WL.max()])
axs[1].axis([0, Z.max(), 4.1*plot_WT.min(), 1.1*plot_WT.max()])
for ax in axs:
    ax.tick_params(labelsize=14)
for ax in axs:
    ax.set_ylabel(r"$W(Z)$", fontsize=14)
    ax.set_xlabel(r"$Z$ (mm)", fontsize=14)
fig.subplots_adjust(wspace=.275)